In [10]:
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [56]:
# -*- coding: utf-8 -*-
"""
Created on Wed Jul  3 06:45:38 2019

@author: night
"""
from collections import defaultdict
import pandas as pd
import networkx as nx
import itertools
import matplotlib.pyplot as plt
import heapq

import graphviz

def get_genre_list(genre_str):
    import re
    p = re.compile(r"'([A-Za-z &]+)'")
    return p.findall(genre_str)

def create_genre_groups(artist_info):
    genre_ct = defaultdict(int)
    genre_groups = []
    
    for i in range(len(artist_info)):
        genre_str = artist_info.iloc[i]['genres']
        genre_list = get_genre_list(genre_str)
        genre_groups.append(genre_list)
        
        for genre in genre_list:
            genre_ct[genre] += 1        
    #sort_dict = sorted(genre_ct.items(), key=lambda item: item[1], reverse = True)
    return genre_groups

def create_graph(genre_groups):
    G = nx.MultiGraph()

    for artist_genres in genre_groups:
        G.add_nodes_from(artist_genres)
        for comb in itertools.combinations(artist_genres, 2):
            G.add_edge(comb[0], comb[1])
    
    S = nx.Graph()
    for u,v,data in G.edges(data=True):
        w = data['weight'] if 'weight' in data else 1.0
        if S.has_edge(u,v):
            S[u][v]['weight'] += w
        else:
            S.add_edge(u, v, weight=w)
    return S

def sort_nodes(G):
    degree_dict = {}
    for (a, b) in G.degree():
        degree_dict[a] = b
    heap = [(-value, key) for key,value in degree_dict.items()]
    degree_largest = heapq.nsmallest(100, heap)
    return [key for value, key in degree_largest]

def sort_edges(G):
    weight_dict = {}
    for (a, b, w) in G.edges.data('weight'):
        weight_dict[(a,b)] = w
    heap = [(-value, key) for key,value in weight_dict.items()]
    weight_largest = heapq.nsmallest(100, heap)
    #weight_largest = [(key, -int(value)) for value, key in weight_largest]
    return [key for value, key in weight_largest]

def plot_network_graph(G, top_num, sorted_nodes):
    bignodes=degree_largest[:20]
    G_small = G.subgraph(bignodes)
    pos_small = nx.nx_pydot.graphviz_layout(G_small)
    nx.draw(G_small, pos=pos_small, node_size=50, with_labels=True)
    return G_small

artist_info = pd.read_csv('data/180806_artistdf.csv')
genre_groups = create_genre_groups(artist_info)
G = create_graph(genre_groups)

# nx.readwrite.graphml.write_graphml(S,'data/genregraph.graphml')

weight_largest = sort_edges(G)
degree_largest = sort_nodes(G)
pos = nx.spring_layout(G)
# G_small = plot_network_graph(S, 100, degree_largest)


In [29]:
import networkx as nx

from bokeh.io import show, output_file
from bokeh.plotting import figure
from bokeh.models.graphs import from_networkx

plot = figure(x_range=(-1.1,1.1), y_range=(-1.1,1.1),
              tools="", toolbar_location=None)

graph = from_networkx(G, nx.spring_layout, scale=6, center=(0,0))
plot.renderers.append(graph)
show(plot)

In [57]:
for (g1,g2) in list(G.edges):
    if G.get_edge_data(g1,g2)['weight']<3:
        G.remove_edge(g1,g2)

In [86]:
import math
nodenames = list(G.nodes())
nodedegree = [genre[1]/3 for genre in list(G.degree)]
weights = [1*math.log(G.get_edge_data(g1,g2)['weight'], 2) for (g1, g2) in list(G.edges)]


In [89]:
from bokeh.io import show, output_file
from bokeh.models import Plot, Range1d, MultiLine, Circle, HoverTool, TapTool, WheelZoomTool, PanTool
from bokeh.models.graphs import from_networkx, NodesAndLinkedEdges, EdgesAndLinkedNodes
from bokeh.palettes import Spectral4, Plasma10
from bokeh.transform import linear_cmap
# from bokeh.embed import json_item
import json

plot = Plot(plot_width=1000, plot_height=700,
            x_range=Range1d(-10,10), y_range=Range1d(-10,10))
plot.title.text = "Graph Interaction Demonstration"



graph_renderer = from_networkx(G, nx.spring_layout, scale=80, center=(0,0))

graph_renderer.node_renderer.data_source.add(nodenames, 'name')
graph_renderer.node_renderer.data_source.add(nodedegree, 'degree')
graph_renderer.edge_renderer.data_source.add(weights, 'weight')

graph_renderer.node_renderer.glyph = Circle(size='degree', fill_color='lightsteelblue', fill_alpha=0.7, line_color='black', line_width=1)
graph_renderer.node_renderer.selection_glyph = Circle(size=15, fill_color=Spectral4[2])
graph_renderer.node_renderer.hover_glyph = Circle(size=15, fill_color=Spectral4[1])

graph_renderer.edge_renderer.glyph = MultiLine(line_color=linear_cmap('weight', 'Plasma10', min(weights), max(weights)), 
                                               line_alpha=1, line_width='weight')
graph_renderer.edge_renderer.selection_glyph = MultiLine(line_color=Spectral4[2], line_width='weight')
graph_renderer.edge_renderer.hover_glyph = MultiLine(line_color=Spectral4[1], line_width='weight')

graph_renderer.selection_policy = NodesAndLinkedEdges()
graph_renderer.inspection_policy = NodesAndLinkedEdges()

plot.renderers.append(graph_renderer)
plot.background_fill_color = "black"
plot.add_tools(HoverTool(tooltips=[('genre', '@name')]), TapTool(), PanTool(), WheelZoomTool())

output_file("interactive_graphs.html")
# item_text = json.dumps(json_item(plot, "myplot"))
show(plot)

NameError: name 'json_item' is not defined

In [50]:
G.nodes.values

<bound method Mapping.values of NodeView(('hip pop', 'neo soul', 'pop', 'r&b', 'urban contemporary', 'bassline', 'dance pop', 'deep pop r&b', 'new jack swing', 'pop rap', 'hip hop', 'rap', 'southern hip hop', 'trap music', 'quiet storm', 'alternative r&b', 'indie r&b', 'funk', 'soul', 'boy band', 'neo mellow', 'new orleans rap', 'atl hip hop', 'dirty south rap', 'edm', 'miami hip hop', 'permanent wave', 'pop rock', 'celtic rock', 'viral pop', 'piano rock', 'indie pop', 'indie poptimism', 'modern rock', 'pop punk', 'dfw rap', 'canadian contemporary r&b', 'canadian pop', 'uk contemporary r&b', 'indie pop rap', 'cali rap', 'east coast hip hop', 'indie rock', 'rock', 'indietronica', 'baroque pop', 'chamber pop', 'rawstyle', 'uk alternative pop', 'indie folk', 'contemporary country', 'country', 'country dawn', 'indonesian jazz', 'electropop', 'uk pop', 'escape room', 'minnesota hip hop', 'trap queen', 'etherpop', 'bachata', 'latin', 'latin pop', 'tropical', 'philly rap', 'tropical house', '

In [80]:
import networkx as nx

from bokeh.io import show, output_file
from bokeh.models import Plot, Range1d, MultiLine, Circle, HoverTool, BoxZoomTool, ResetTool
from bokeh.models.graphs import from_networkx
from bokeh.palettes import Spectral4



# for start_node, end_node, _ in G.edges(data=True):
#     edge_color = SAME_CLUB_COLOR if G.nodes[start_node]["club"] == G.nodes[end_node]["club"] else DIFFERENT_CLUB_COLOR
#     edge_attrs[(start_node, end_node)] = edge_color

# nx.set_edge_attributes(G, edge_attrs, "edge_color")

# Show with Bokeh
plot = Plot(plot_width=700, plot_height=700,
            x_range=Range1d(-10,10), y_range=Range1d(-10,10))
plot.title.text = "Graph Interaction Demonstration"

node_hover_tool = HoverTool(tooltips=[("genre",'@name')])
plot.add_tools(node_hover_tool, BoxZoomTool(), ResetTool())

graph_renderer = from_networkx(G, nx.spring_layout, scale=40, center=(0, 0))

graph_renderer.node_renderer.glyph = Circle(size=7, fill_color=Spectral4[0])
graph_renderer.edge_renderer.glyph = MultiLine(line_alpha=0.8, line_width=1)
plot.renderers.append(graph_renderer)
show(plot)

ERROR:bokeh.core.validation.check:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: edge_color [renderer: GlyphRenderer(id='dd6cfaf9-d047-4287-b061-0165d7fe8048', ...)]
